<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/chart.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Chart　クラス解説

Chartクラスは一般的に使われるohlcvのチャートとインジケーターの描画、およびBackRunnerが保持する結果を表示する機能を提供しています。

Chartクラスは、インタラクティブなグラフが書けるbokehライブラリ上で構築しています。線種などの指定はbokehのパラメータがそのまま使えます。

## コンストラクタ

* `Chart(width, height, ohlcv)`
    - `width`  チャートの幅(pixcel)
    - `height` チャートの高さ(pixcel)
    - `ohlcv`  pd.DataFrame形式のデータ('open', 'high', 'low', 'close', 'volume'のカラム名が必須)

`width`, `height`の指定場所は以下の図を参照。コンストラクタの中では、ohlcvを描画するために`Price`と`Volume`＋スライダーの３つのパネルが作られる。
<img src=https://github.com/yasstake/rusty-bot/blob/main/manual/chart_panel.png?raw=1>



<img src='https://github.com/yasstake/rusty-bot/blob/main/manual/img/chart_panels.png?raw=1'/>

## メソッド

* `show()` チャートの表示。すべての準備が終わった最後に呼び出す。

* `new_figure(name, height, title)`　あたらしい描画パネルを作成する。
  - `name`　作成するパネル名
  - `height`　高さ
  - `title`　タイトル


* `line(figure, df, x_key,  y_key, **kwargs)`　折線グラフを描画する。
  - `figure`　描画したいパネル名 
  - `df`　データ(pd.DataFrame)
  - `x_key` pd.DataFrameの時間軸にあたるカラム名
  - `y_key` pd.DataFrameのy軸で描画したいカラム名
  - `**kwargs` bokehのlineメソッドへ引き継がれるパラメータ。色や線の幅を指定する。


* `step(figure, df, x_key=None, y_key=None, **kwargs)`　ステップグラフ（データ系列の間変化しないグラフ）
  - `figure`　描画したいパネル名
  - `df`　データ(pd.DataFrameまたはpd.Series)
  - `x_key` pd.DataFrameの時間軸にあたるカラム名
  - `y_key` pd.DataFrameのy軸で描画したいカラム名
  - `**kwargs` bokehのlineメソッドへ引き継がれるパラメータ。色や線の幅を指定する。


* `draw_result(df)` BackRunnerが保持する結果を表示する
  - `BackRunner.result`に保存されているpd.DataFrame形式のバックテストの結果データ


### `**kwargs`に指定するパラメータ

Bokehのlineあるいはstepへ直接渡しています。そのため以下にあるパラメータのほとんどが使えます。
https://docs.bokeh.org/en/latest/docs/reference/plotting/figure.html#bokeh.plotting.figure.line

#### 代表的なもの

* `color=`  色: 赤は `#FF0000`のようにRGBで指定します。透明度を指定するときは最後に2桁追加します。
* `legend_label=` 凡例に表示する名前
* `line_width=`　線の幅

# 以下動かしながら説明

## 必要ライブラリのインストール（google collab用）

環境にあわせ適宜ダウンロードしてください。
https://github.com/yasstake/rusty-bot

ここからリリースページへ遷移して環境に合うバイナリを探してください。

In [1]:
# Google Clab用です。必要に応じてコメントアウトしてください
! pip install --upgrade pip
! pip install numpy
! pip install pandas
! pip uninstall -y panel
! pip install --upgrade bokeh >= 3

# https://github.com/yasstake/rusty-bot/releases/tag/release-0.2.0
# （環境にあわせて修正お願いします）
# Linux/Google Collab用
!  pip install https://github.com/yasstake/rusty-bot/releases/download/release-0.2.0a/rbot-0.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.4 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: panel 0.12.1
Uninstalling panel-0.12.1:
  Successfully uninstalled panel-0.12.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 15.6 MB/s eta 0:00:00


## データのダウンロード＆OHLCVデータの作成

In [2]:
from rbot import Market
from rbot import DAYS_BEFORE

#　3日分のデータのダウンロード。　以後このohlcvをChartクラスで表示していく
binance = Market.open("BN", "BTCBUSD")
Market.download(3)

ohlcv = binance.ohlcv(DAYS_BEFORE(3), 0, 60)
ohlcv

,open,high,low,close,volume
timestamp,,,,,
2022-12-19 06:24:00+00:00,16716.28,16716.71,16710.10,16711.68,52.29348
2022-12-19 06:25:00+00:00,16711.81,16713.15,16707.18,16707.31,33.24807
2022-12-19 06:26:00+00:00,16707.34,16710.95,16704.65,16707.28,39.31204
2022-12-19 06:27:00+00:00,16707.93,16719.65,16706.20,16718.59,41.44804
2022-12-19 06:28:00+00:00,16719.17,16719.55,16713.77,16714.82,24.80204
...,...,...,...,...,...
2022-12-21 23:55:00+00:00,16802.69,16810.41,16801.24,16808.96,46.72673
2022-12-21 23:56:00+00:00,16810.36,16816.02,16809.55,16814.21,64.08570
2022-12-21 23:57:00+00:00,16814.54,16816.40,16809.24,16812.91,41.66195


## チャートの作成

### シンプルチャート

In [3]:
# 必要パッケージのimport
import rbot
from rbot.chart import Chart

# Chartクラスの作成
chart = Chart(
    900,    # 横幅
    400,    # Price表示エリアの縦幅
    ohlcv   # 表示するohlcv. 'open', 'high', 'low', 'close', 'volume'があれば表示できる。
)
chart.show()

## 移動平均線の追加

pandasの`rolling`関数を使って移動平均線を加えてみよう。

＊本格的な指標作成には、TA-LIBなど使ったほうがいいと思いますが、ここは説明のため一番簡単な方法で進めます。


In [11]:
import pandas as pd
# 5本＝５分足の移動平均
ma5 = pd.DataFrame(ohlcv.rolling(5)['close'].mean())

# 60本＝１時間の移動平均
ma60 = pd.DataFrame(ohlcv.rolling(60)['close'].mean())
ma60

,close
timestamp,
2022-12-19 06:24:00+00:00,NaN
2022-12-19 06:25:00+00:00,NaN
2022-12-19 06:26:00+00:00,NaN
2022-12-19 06:27:00+00:00,NaN
2022-12-19 06:28:00+00:00,NaN
...,...
2022-12-21 23:55:00+00:00,16789.466333
2022-12-21 23:56:00+00:00,16790.231000
2022-12-21 23:57:00+00:00,16790.973833


### ohlcvチャートの上に移動平均を書く

ohlcvチャートが書かれているパネルは`price`なのでここに`line`メソッドで描画する。

In [13]:
chart = Chart(
    900,    # 横幅
    400,    # Price表示エリアの縦幅
    ohlcv   # 表示するohlcv. 'open', 'high', 'low', 'close', 'volume'があれば表示できる。
)

chart.line('price', ma5, x_key='timestamp', y_key='close', legend_label='ma5', color='#ff000080', line_width=3)
chart.line('price', ma60, x_key='timestamp', y_key='close', legend_label='ma60', color='#00ffff80', line_width=2)

chart.show()


### 新しいパネルを作って書く方法

`new_figure`で新しいパネルを作り、作ったパネル名を`line`の引数にいれてグラフを描画する。

In [14]:
chart = Chart(
    900,    # 横幅
    400,    # Price表示エリアの縦幅
    ohlcv   # 表示するohlcv. 'open', 'high', 'low', 'close', 'volume'があれば表示できる。
)

# maという名前で、高さ200の新しい描画パネルを作成する。タイトルとしてmovig averageと表示する。
chart.new_figure(name='ma', height=200, title='moving average')

#chart.new_figure('p2', 100,'price2')
chart.line('ma', ma5, x_key='timestamp', y_key='close', legend_label='ma5', color='#ff000080', line_width=3)
chart.line('ma', ma60, x_key='timestamp', y_key='close', legend_label='ma60', color='#00ffff80', line_width=2)

chart.show()


## まとめ

Chartクラスは全体から独立して使うことができます。
見た目がいまいちかもしれないけれど、かなりシンプルに使うことができます。

見た目をカストマイズしたい場合はパラメータ設定を工夫するといいでしょう。以下にソースを添付しますので、ソースから修正してもいいと思います。

# ソースコード

ソースはPythonのsite-package以下にインストールされているはずですが、いちいち開くのも大変ですので以下にコピーしておきました。見た目をかえるときに参考にしてパラメータ設定すると便利です。

さらに、このChartクラスは好きに使ってOKです。大したもんじゃないので、コピーとかして自分のプロダクトに組み込んでOK。ただし無保証・無責任。

In [ ]:
from collections import OrderedDict
import numpy as np
import pandas as pd

from bokeh.layouts import column 
from bokeh.models import ColumnDataSource, RangeTool, HoverTool, CrosshairTool, Span
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
import datetime

class Chart:
    def __init__(self, width, height, ohlcv):
        output_notebook()        
        
        self.figure = OrderedDict()
        self.width = width
        self.x_range = None

        ######### create main price figure ############
        TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

        # setup main figure
        dates = np.array(ohlcv.index, dtype=datetime.datetime)        
        price = figure(x_axis_type="datetime", tools=TOOLS, width=self.width, height=height,
           title="Price", background_fill_color="#efefef", x_range=(dates[0], dates[-1]))
        
        self.x_range = price.x_range
        self.figure['price'] = price
        
        self.draw_ohlc(price, ohlcv)

        span_height = Span(dimension="height", line_dash="dashed", line_width=1)
        self.cross_hair = CrosshairTool(overlay=span_height)        
        price.add_tools(self.cross_hair)

        ########  create volume figure ################
        if 'volume' in ohlcv.column:
            volume = self.new_figure('Volume', 100, 'Volume')
            self.draw_volume('Volume', ohlcv)
            volume.add_tools(self.cross_hair)

        ######### setup select figure #################
        select = figure(title="Price slide bar",
                height= int(height/4), width=self.width, y_range=price.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")            

        self.select = select

        self.line(select, ohlcv, x_key='timestamp', y_key='close', legend_label='price', color='#1010ff')
        range_tool = RangeTool(x_range=self.x_range)
        range_tool.overlay.fill_color = "navy"
        range_tool.overlay.fill_alpha = 0.2

        select.ygrid.grid_line_color = None
        select.add_tools(range_tool)
        select.toolbar.active_multi = range_tool
        select.add_tools(self.cross_hair)
        
    
    def new_figure(self, name, height, title):
        p = figure(x_axis_type="datetime", width=self.width, height=height, tools="", toolbar_location=None,
            title=title, background_fill_color="#efefef", x_range=self.x_range)
        self.figure[name] = p
        p.add_tools(self.cross_hair)

        return p 
        
    def get_figure(self, figure):
        if isinstance(figure, str):
            if figure in self.figure:
                return self.figure[figure]
        
            return self.new_figure(figure, 100, figure)
        else:
            return figure

    def show(self):
        figure = []
        for key in self.figure:
            figure.append(self.figure[key])

        figure.append(self.select)
            
        show(column(figure))

    def draw_result(self, df):
        buy_df = df[(df['order_side'] == "Buy") & (df['status'] != 'Expire')]
        buy_df_e = df[(df['order_side'] == "Buy") & (df['status'] == 'Expire')]
        sell_df = df[(df['order_side'] == "Sell") & (df['status'] != 'Expire')]
        sell_df_e = df[(df['order_side'] == "Sell") & (df['status'] == 'Expire')]

        p = self.new_figure('profit', 150, 'profit')
        self.step(p, df, 'update_time', 'sum_profit', legend_label='profit', color="#ff8080")

        p = self.new_figure('position', 100, 'position')
        self.step(p, df, 'update_time', 'position', legend_label='position', color="#ff8080")

        p = self.get_figure('price')
        self.draw_order_maker(p, buy_df, 'triangle', fill_color="#00ff00", line_color="#00ff00", legend_name='Buy')
        self.draw_order_maker(p, sell_df, 'inverted_triangle', fill_color="#ff0000", line_color="#ff0000", legend_name='Sell')
        self.draw_order_maker(p, buy_df_e, 'triangle', fill_color="#00000000", line_color="#00ff00", legend_name='Buy Expire')
        self.draw_order_maker(p, sell_df_e, 'inverted_triangle', fill_color="#00000000", line_color="#ff0000", legend_name='Sell Expire')


    def draw_ohlc(self, p, ohlc):
        ds = ColumnDataSource(ohlc)

        df_inc = ColumnDataSource(ohlc[(ohlc['open'] <= ohlc['close'])])
        df_dec = ColumnDataSource(ohlc[(ohlc['close'] < ohlc['open'])])

        delta = (ohlc[1:2].index - ohlc[0:1].index)[0]
        w = delta.total_seconds() * 1_000 * 0.8

        p.segment('timestamp', 'high', 'timestamp', 'low', source=ds, color="#080808")
        vbar_dec = p.vbar('timestamp', w, 'close', 'open', source=df_dec, fill_color='#ff66ff', line_color='#ff0000', line_width=0)        
        vbar_inc = p.vbar('timestamp', w, 'open', 'close', source=df_inc, fill_color='#66ccff', line_color="#10ff80", line_width=0)

        hover_inc = HoverTool(
            renderers=[vbar_inc],
            tooltips = [
                ("timestamp", "@timestamp{%F %R.%S}"),
                ("open", "@open{0.0}"),
                ("high", "@high{0.0}"),
                ("low", "@low{0.0}"),
                ("close", "@close{0.0}")
            ],
            formatters= {
                "@timestamp": "datetime",
            },
            mode="vline",
            show_arrow=False,
        )       

        hover_dec = HoverTool(
            renderers=[vbar_dec],
            tooltips = [
                ("timestamp", "@timestamp{%F %R.%S}"),
                ("open", "@open{0.0}"),
                ("high", "@high{0.0}"),
                ("low", "@low{0.0}"),
                ("close", "@close{0.0}")
            ],
            formatters= {
                "@timestamp": "datetime"
            },
            show_arrow=False,
        )      

        p.add_tools(hover_inc)
        p.add_tools(hover_dec)


    def draw_order_maker(self, p, df, marker, fill_color, line_color, legend_name, **kwargs):
        df = ColumnDataSource(df)
        scatter = p.scatter(x='update_time', y='order_price', source=df, marker=marker, size=12, fill_color=fill_color, line_color=line_color, line_width=1, legend_label=legend_name, **kwargs)

        hover = HoverTool(
            renderers=[scatter],
            tooltips = [
                (legend_name,""),
                ("timestamp", "@update_time{%F %R.%S}"),
                ("Price", "@order_price{0.0}")
            ],
            formatters={
                "@update_time": "datetime"
            }
        )
        p.add_tools(hover)    

    def draw_volume(self, figure, ohlcv):
        p = self.get_figure(figure)        
        self.line(p, ohlcv, x_key='timestamp', y_key='volume', color='#00ffff', legend_label='volume')

    def line(self, figure, df, x_key= None, y_key=None, **kwargs):
        p = self.get_figure(figure)
        df = self.make_df_from_series(df)        
        
        if not x_key:
            x_key = 'timestamp'
        
        if not y_key:
            y_key = 'value'

        p.line(x=x_key, y=y_key, source=df, **kwargs)

    
    def step(self, figure, df, x_key=None, y_key=None, **kwargs):
        p = self.get_figure(figure)
        df = self.make_df_from_series(df)

        if not x_key:
            x_key = 'timestamp'
        
        if not y_key:
            y_key = 'value'

        p.step(x=x_key, y=y_key, source=df, mode='after', **kwargs)

    def make_df_from_series(self, df):
        return ColumnDataSource(self.make_df(df))
    
    def make_df(self, df):
        if isinstance(df, pd.DataFrame):
            df = df
        elif isinstance(df, pd.Series):
            df = df.rename('value')
            df = pd.DataFrame(df)
            df.index.name = 'timestamp'

            df.index = pd.to_datetime(df.index, utc=True, unit='us')
            df = df.dropna()
    
        df = df[~df.index.duplicated(keep='last')]
        
        return df



